# Modeling Machine Learning Deteksi Kardus

Terdiri delapan satu kelas kue yaitu kardus dengan dataset pelatihan berupa 300 gambar

Training model dibuat menggunakan algoritma Yolo V3 Tiny

In [ ]:
# Cek NVIDIA CUDA
!nvidia-smi

Wed Oct 19 06:52:41 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Instalasi dan Konfigurasi Darknet

In [ ]:
# Clone Darknet
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15494, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 15494 (delta 0), reused 1 (delta 0), pack-reused 15490
Receiving objects: 100% (15494/15494), 14.09 MiB | 22.86 MiB/s, done.
Resolving deltas: 100% (10411/10411), done.


In [ ]:
# Konfigurasi Darkent untuk penggunaan Algoritma Yolov3
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

# Compile Konfigurasi Darknet
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

## Konfigurasi File Model Yolov3.cfg

In [ ]:
# Copy file yolov3.cfg untuk digunakan dalam model
!cp cfg/yolov3-tiny.cfg cfg/yolov3_training.cfg

# Mengubah beberapa parameter layer untuk penyesuaian model yang lebih baik
# max_batches = kelas * 2000
# classes = kelas
# filters = (kelas + 5) * 3
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 10000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

##Membuat Direktori Data Training

In [ ]:
# Membuat file direktori data training
!echo -e 'Kardus' > data/obj.names
!echo -e 'classes= 1\ntrain= data/train.txt\nvalid= data/test.txt\nnames= data/obj.names\nbackup= /content/gdrive/MyDrive/yolov3tiny' > data/obj.data

In [ ]:
# Save model konfigurasi yolov3_training.cfg 
!cp data/obj.names /content/gdrive/MyDrive/yolov3tiny/classes.txt

In [ ]:
!mkdir data/obj

In [ ]:
# Unzip Dataset Gambar Kardus
!unzip /content/gdrive/MyDrive/yolov3tiny/Dataset_Pelatihan.zip -d data/obj

Archive:  /content/gdrive/MyDrive/yolov3tiny-update/Dataset_Pelatihan.zip
   creating: data/obj/Dataset_Pelatihan/
  inflating: data/obj/Dataset_Pelatihan/classes.txt  
  inflating: data/obj/Dataset_Pelatihan/IMG_20221012_125649.jpg  
  inflating: data/obj/Dataset_Pelatihan/IMG_20221012_125649.txt  
  inflating: data/obj/Dataset_Pelatihan/IMG_20221012_125653.jpg  
  inflating: data/obj/Dataset_Pelatihan/IMG_20221012_125653.txt  
  inflating: data/obj/Dataset_Pelatihan/IMG_20221012_125700.jpg  
  inflating: data/obj/Dataset_Pelatihan/IMG_20221012_125700.txt  
  inflating: data/obj/Dataset_Pelatihan/IMG_20221012_125706.jpg  
  inflating: data/obj/Dataset_Pelatihan/IMG_20221012_125706.txt  
  inflating: data/obj/Dataset_Pelatihan/IMG_20221012_125711.jpg  
  inflating: data/obj/Dataset_Pelatihan/IMG_20221012_125711.txt  
  inflating: data/obj/Dataset_Pelatihan/IMG_20221012_125717.jpg  
  inflating: data/obj/Dataset_Pelatihan/IMG_20221012_125717.txt  
  inflating: data/obj/Dataset_Pelatihan

In [ ]:
import glob
images_list = glob.glob("data/obj/Dataset_Pelatihan/*.jpg")
with open("data/train.txt", "w") as f:
    f.write("\n".join(images_list))

In [ ]:
#Donwload pre-trained weights untuk mengambil layer konvolusi
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2022-10-19 07:03:26--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  45.2MB/s    in 3.8s    

2022-10-19 07:03:31 (40.8 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



## Training Model

In [ ]:
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show
# !./darknet detector train data/obj.data cfg/yolov3_training.cfg /mydrive/yolov3tiny/yolov3_training_last.weights -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 23 Avg (IOU: 0.000000), count: 1, class_loss = 0.000003, iou_loss = 0.000000, total_loss = 0.000003 
 total_bbox = 57715, rewritten_bbox = 0.000000 % 
]2;903/1000: loss=1.0 hours left=0.4
 903: 1.040734, 0.865354 avg loss, 0.000665 rate, 0.945906 seconds, 57792 images, 0.394251 hours left
Loaded: 5.902733 seconds - performance bottleneck on CPU or Disk HDD/SSD
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 16 Avg (IOU: 0.564397), count: 4, class_loss = 2.002608, iou_loss = 0.386186, total_loss = 2.388794 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 23 Avg (IOU: 0.000000), count: 1, class_loss = 0.000003, iou_loss = 0.000000, total_loss = 0.000003 
 total_bbox = 57719, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 16 Avg (IOU: 0.534974), count: 4, class_loss = 2.088738, iou_los